In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
import seaborn as sns
%matplotlib inline

Place the exported chat file in the same directory and change the file name below

In [ ]:
group_name="BOIOIOING BOIS"
exported_file="boioioing.txt"
chats=open(exported_file,"r")

In [ ]:
df=pd.DataFrame(columns=["Day","Month","Year","Hour","Minutes","Sender","Message"])

Following block parses text file and creates a pandas data frame

In [ ]:
i=0
while True:
    line=chats.readline()
    if not line:
        break
    x=re.search("(([0-9]){2}/){2}([0-9]){4}, [0-9]{2}:[0-9]{2} - ",line)  
    y=re.search("(([0-9]){2}/){2}([0-9]){4}, [0-9]{2}:[0-9]{2} - .+?:",line)  
    
    if x is None:
        if i!=0:
            df.loc[i-1][6]+=line
            continue
        else:
            print("First line doesnt contain a timestamp")
            continue
    else:
        targetString=x.group()
        if (re.search("changed the subject from",line) is None) and (re.search("this group's icon",line) is None):
            if y is not None:
                nameString=y.group()
                df.loc[i]=[targetString[0:2],targetString[3:5],targetString[6:10],targetString[12:14],targetString[15:17],nameString[20:-1],line[y.span()[1]:-1]]
            else:
                df.loc[i]=[targetString[0:2],targetString[3:5],targetString[6:10],targetString[12:14],targetString[15:17],np.nan,line[x.span()[1]:-1]]
        else:
            df.loc[i]=[targetString[0:2],targetString[3:5],targetString[6:10],targetString[12:14],targetString[15:17],np.nan,line[x.span()[1]:-1]]
    i+=1


In [ ]:
if "Timestamp" not in df:
    df.insert(0,"Timestamp",pd.to_datetime(df["Day"]+"-"+df["Month"]+"-"+df["Year"]+" "+df["Hour"]+":"+df["Minutes"],dayfirst=True),True)
df=df.set_index('Timestamp')

In [ ]:
print(i, " chats read")
df.head()

In [ ]:
print("Name/Phone\tMessages Sent")
message_counts=df["Sender"].value_counts(sort=True,dropna=True)
print(message_counts)
user_list=message_counts.index.tolist()
message_count_list=message_counts.values.tolist()

In [ ]:
plt.title("Distribution of messages",y=1.08)
labels=message_counts.index.tolist()[:5]
reduced_message_counts=message_count_list[:5]
if sum(message_count_list[5:])!=0:
    reduced_message_counts.append(sum(message_count_list[5:]))
    
    labels.append("Others")
plt.pie(reduced_message_counts,autopct='%1.1f%%',pctdistance=1.2,labels=labels,labeldistance=1.4)

plt.savefig(group_name+" message distribution.png")
plt.show()

In [ ]:
df.head(15)

In [ ]:
avg_message_lengths=[]
for i in user_list:
    avg_message_lengths.append(df[df["Sender"]==i]["Message"].str.split().apply(len).mean())

In [ ]:
sns.set()
pos=np.arange(len(user_list))*0.3
plt.bar(pos, avg_message_lengths, width=0.1,align='center', alpha=0.9)
plt.xticks(pos, user_list)
plt.ylabel('Average number of words in a message')
plt.savefig(group_name+" average words.png")
plt.show()



In [ ]:
sns.set(rc={'figure.figsize':(11, 4)})
df["Message"].apply(len).plot(linewidth=0.5);